In [1]:
#Import useful functions
import warnings
warnings.filterwarnings("ignore")
from sklearn import svm, datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix as cm
from sklearn.utils.multiclass import unique_labels
import itertools
from scipy import misc
import math
import os
import glob
import xlrd
import xlwt
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
from PIL import Image
from statsmodels.tsa.statespace.sarimax import SARIMAX
import statsmodels.api as sm
import pylab as plt
import numpy as np
from scipy.signal import argrelextrema
from IPython.display import Image
import matplotlib.patches as patches
import matplotlib as matplotlib
import xlsxwriter
from xlrd import open_workbook
from sklearn.model_selection import train_test_split
from datetime import datetime
plt.style.use('fivethirtyeight')
import itertools
from scipy.interpolate import interp1d
from sklearn.preprocessing import StandardScaler
import xlrd

In [2]:
def sumorders(data, colstr, datastr):
    data=data[data[colstr]==datastr]
    data=data.groupby(['SKU'])['Amount', 'Quantity'].sum().reset_index()
    newdata=data.set_index('SKU')
    return newdata

def sumexpenses(filename, sheetname, index, index2):
    data=pd.read_excel(filename, sheet_name=sheetname)
    data=data.groupby([index, index2]).sum().reset_index()
    newdata=data.set_index(index)
    return newdata

In [3]:
def createindex(filename, plpd):
    orders1=pd.read_excel(filename, sheet_name='Orders & FBA Commissions')
    orders=orders1.groupby(['SKU', 'Product Title']).sum().reset_index()
    ordersaddor=pd.read_excel(filename, sheet_name='Monthly SKU Sales')
    ordersaddraw=ordersaddor.groupby(['(Child) ASIN', 'SKU']).sum().reset_index()
    ordersaddasin=ordersaddraw.set_index('(Child) ASIN')
    ordersadd=ordersaddraw.set_index('SKU')
    ordersaddtit=ordersaddor.groupby([ 'SKU', 'Title']).sum().reset_index()
    ordersaddtit=ordersaddtit.set_index('SKU')
    inventoryraw=pd.read_excel(filename, sheet_name='Storage Fees')
    inventory=inventoryraw.groupby(['asin', 'fnsku', 'product-name'])['estimated-monthly-storage-fee'].sum().reset_index()
    invadd=pd.read_excel(filename, sheet_name='Monthly Inventory')
    invaddor=invadd.groupby(['sku', 'fnsku']).sum().reset_index()
    invadd=invaddor.set_index('fnsku')
    invadd2=invaddor.set_index('sku')
    adsraw=pd.read_excel(filename, sheet_name='Ad Spend')
    ads=adsraw.groupby(['Advertised SKU', 'Advertised ASIN']).sum().reset_index()
    ads=ads.set_index('Advertised SKU')
    refund1=pd.read_excel(filename, sheet_name='Refunds')
    refund=refund1.groupby(['SKU', 'Product Title']).sum().reset_index()
    refundadord=refund1.groupby(['Order ID', 'SKU']).sum().reset_index()



    for ind in inventory.index:
        asin=inventory.loc[ind, 'asin']
        title=inventory.loc[ind, 'product-name']
        fnsku=inventory.loc[ind, 'fnsku']
        if asin not in plpd['ASIN'].values:
            if fnsku in invadd.index:
                sku=invadd.loc[fnsku, 'sku']
                inventory.loc[ind, 'sku']=sku
                plpd=plpd.append({'SKU': sku, 'ASIN': asin, 'Product Title': title}, ignore_index=True) 
            elif asin in ordersaddasin.index:
                sku=ordersaddasin.loc[asin, 'SKU']
                inventory.loc[ind, 'sku']=sku
                plpd=plpd.append({'SKU': sku, 'ASIN': asin, 'Product Title': title}, ignore_index=True)
            elif title in ordersaddtit['Title'].values:
                sku=ordersaddtit[ordersaddtit['Title'].values==title, 'SKU']
                inventory.loc[ind, 'sku']=sku
                plpd=plpd.append({'SKU': sku, 'ASIN': asin, 'Product Title': title}, ignore_index=True)
            else:
                plpd=plpd.append({'SKU': 'unknown', 'ASIN': asin, 'Product Title': title}, ignore_index=True)
    for ind in orders.index:
        sku=orders.loc[ind, 'SKU']
        title=orders.loc[ind, 'Product Title']
        if sku not in plpd['SKU'].values:
            if sku in ordersadd.index:
                asin=ordersadd.loc[sku, '(Child) ASIN']
                plpd=plpd.append({'SKU': sku, 'ASIN': asin, 'Product Title': title}, ignore_index=True)
            elif sku in ads.index:
                asin=ads.loc[sku, 'Advertised ASIN']
                plpd=plpd.append({'SKU': sku, 'ASIN':asin, 'Product Title': title}, ignore_index=True)
            else:  
                plpd=plpd.append({'SKU': sku, 'ASIN': 'Unknown', 'Product Title': title}, ignore_index=True)
    for ind in adsraw.index:
        sku=adsraw.loc[ind, 'Advertised SKU']
       
        if sku not in plpd['SKU'].values:
            if sku in ordersaddtit.index:
                title=ordersaddtit.loc[sku, 'Title']
                asin=ordersadd.loc[sku, '(Child) ASIN']
                plpd=plpd.append({'SKU': sku, 'ASIN': asin, 'Product Title': title}, ignore_index=True)
            else:
                plpd=plpd.append({'SKU': sku, 'ASIN': asin, 'Product Title': 'Unknown'}, ignore_index=True)
    for ind in refund.index:
        sku=refund.loc[ind, 'SKU']
        title=refund.loc[ind, 'Product Title']
        order=refund.loc
        if sku not in plpd['SKU'].values:
            if sku in ordersadd.index:
                asin=ordersadd.loc[sku, '(Child) ASIN']
                plpd=plpd.append({'SKU': sku, 'ASIN': asin, 'Product Title': title}, ignore_index=True)
            else:
                plpd=plpd.append({'SKU': sku, 'ASIN': 'Unknown', 'Product Title': title}, ignore_index=True)
    storage=inventory.groupby('sku')['estimated-monthly-storage-fee'].sum().reset_index()
    storage=storage.set_index('sku')
        
    return plpd,storage, orders

In [4]:
paydet=['Sales Tax Service Fee','Referral Fee on Item Price','FBA Pick & Pack Fee', 'Shipping Chargeback', 'Gift Wrap Chargeback', 'Closing Fee',
        'Shipping', 'Gift wrap', 'Promo rebates', 'Refund Administration Fee', 'FBA Customer Return Per Unit Fee', 'Product charges', 'Shipping tax', 'Gift Wrap Tax']
def simplifycode( filename, sheetname, paydet, feenum):
    orders=pd.read_excel(filename, sheet_name='Orders & FBA Commissions')
    orderidad=orders.groupby(['Order ID', 'SKU']).sum().reset_index()
    orderidad=orderidad.set_index('Order ID')
    orders=pd.read_excel(filename, sheet_name=sheetname)
    for ind in orders.index:
        sku=orders.loc[ind, 'SKU']
        order=orders.loc[ind, 'Order ID']
        if str(sku)[0]=='n' or str(sku)[0]=='N':
            if order in orderidad.index:
                newsku=orderidad.loc[order, 'SKU']
                orders.loc[ind, 'SKU']=newsku
            else:
                orders.loc[ind, 'SKU']='Other concession'
                
    sales=sumorders(orders, 'Payment Type', 'Product charges')

    taxfee=sumorders(orders, 'Payment Detail', paydet[0])
    restock=sumorders(orders, 'Payment Detail', 'Restocking fee')
    ref=sumorders(orders, 'Payment Detail', paydet[1])
    shipcom=sumorders(orders, 'Payment Detail','Shipping commission')
    shipback=sumorders(orders, 'Payment Detail', paydet[3])
    giftback=sumorders(orders, 'Payment Detail', paydet[4])
    other=orders[orders['Payment Type']=='Other']
    shipping=sumorders(other, 'Payment Detail', paydet[6])
    closefee=sumorders(orders, 'Payment Detail', paydet[5])
    gift=sumorders(orders, 'Payment Detail', paydet[7])
    promoorig=orders[orders['Payment Type']=='Promo rebates']
    promo=promoorig.groupby(['SKU'])['Amount', 'Quantity'].sum().reset_index()
    promo=promo.set_index('SKU')
    promoship=sumorders(promoorig, 'Payment Detail', paydet[6])
    fbareturnfee=sumorders(orders, 'Payment Detail', paydet[10])
    fbapay=sumorders(orders, 'Payment Detail', paydet[feenum])
    shiptax=sumorders(orders, 'Payment Detail', paydet[12])
    gifttax=sumorders(orders, 'Payment Detail', paydet[13])
    
    return sales, taxfee, ref, shipback, giftback, shipping, gift, promo, promoship, fbareturnfee, fbapay, shiptax, gifttax, shipcom, restock

In [5]:
def replacereverse(filename):
    replacements=pd.read_excel(filename, sheet_name='Replacements')
    replace=replacements.groupby(['sku'])['quantity'].sum().reset_index()
    replace=replace.set_index('sku')
    adspend=pd.read_excel(filename, sheet_name='Ad Spend')
    adspend=adspend.groupby(['Advertised SKU'])['Spend'].sum().reset_index()
    adspend=adspend.set_index('Advertised SKU')
    reverse=pd.read_excel(filename, sheet_name='Reversal Reimbursement')
    invdamage=reverse[reverse['reason']!='Reimbursement_Reversal']
    invdamage=invdamage.groupby(['sku'])['amount-total'].sum().reset_index()
    invdamage=invdamage.set_index('sku')
    reversal=reverse[reverse['reason']=='Reimbursement_Reversal']
    reversal=reversal.groupby(['sku'])['amount-total'].sum().reset_index()
    reversal=reversal.set_index('sku')
 #   brand=pd.read_excel(filename, sheet_name='Brand Spend')
   # brandspend=np.sum(brand['Spend'])
    brandspend=0
    return replace, adspend , invdamage, reversal, brandspend

In [6]:
def servicefees(filename, plpd):
    orders=pd.read_excel(filename, sheet_name='Orders & FBA Commissions')
    service=orders[orders['Transaction type']=='Service Fees']
    
    for ind in service.index:
        pay=service.loc[ind, 'Payment Detail']
        if str(pay)[0:6]=='Coupon':
            service.loc[ind, 'Payment Detail']='Coupon Redemption Fees'
        elif str(pay)[0:12]=='Early Review':
            service.loc[ind, 'Payment Detail']='Early Reviewer Program'
    service=service.groupby(['Payment Detail'])['Amount'].sum().reset_index()
    service=service.set_index('Payment Detail')
    
    adcosttot=0
    totstore=0
    totltstore=0
    invfee=0
    subfee=0
    inbound=0
    for fee in service.index:
        if  fee=='Cost of Advertising' :
            adcosttot=service.loc[fee, 'Amount']+adcosttot


        if  fee== 'FBA Removal Order: Removal Fee':
            invfee=service.loc[fee, 'Amount']+invfee
        if  fee=='FBA Removal Order: Return Fee':
            invfee=service.loc[fee, 'Amount']+invfee
        if  fee== 'FBA Removal Order: Disposal Fee':
            invfee=service.loc[fee, 'Amount']+invfee
        if  fee=='FBA Customer Return Per Unit Fee':
            invfee=service.loc[fee, 'Amount']+invfee
        if fee=='FBA Prep Fee: Polybagging':
            invfee=service.loc[fee, 'Amount']+invfee
        if fee=='FBA Prep Fee: Labeling':
            invfee=service.loc[fee, 'Amount']+invfee
        if fee=='FBA Inventory Placement Service Fee':
            invfee=service.loc[fee, 'Amount']+invfee

        if fee=='FBA Inventory Storage Fee':
            totstore=service.loc[fee, 'Amount']+totstore
        if fee== 'FBA Long-Term Storage Fee' :
            totltstore=service.loc[fee, 'Amount']+totltstore
   

        if fee=='FBA Inbound Transportation Charge':
            inbound=service.loc[fee, 'Amount']+inbound
        if fee=='FBA Amazon-Partnered Carrier Shipment Fee':
            inbound=service.loc[fee, 'Amount']+inbound


        if fee=='Coupon Redemption Fees' :
            subfee=service.loc[fee, 'Amount']+subfee
        if fee== 'Premium Services Fee':
            subfee=service.loc[fee, 'Amount']+subfee
        if fee=='Subscription':
            subfee=service.loc[fee, 'Amount']+subfee
        if fee=='Early Review' :
            subfee=earlyfee.loc[fee, 'Amount']+subfee
            
    plpd.loc['Payment Report Total', 'Inventory Storage Fees']=float(totstore+totltstore)  
    plpd.loc['Payment Report Total',  'Inventory Placement, Prep & Removal Fees']=invfee
    plpd.loc['Payment Total',  'Inventory Placement, Prep & Removal Fees']=invfee
    
    plpd.loc['Payment Report Total','Inbound Carrier Shipping Fees']=inbound
    plpd.loc['Payment Total','Inbound Carrier Shipping Fees']=inbound
    print(totstore)
    plpd.loc['Payment Report Total', 'Advertising Spend']=adcosttot
    plpd.loc['Payment Report Total','Subscription & Service Fees']=subfee
    plpd.loc['Payment Total','Subscription & Service Fees']=subfee
  
    return plpd, totltstore

In [7]:
folder =  '/Users/JoyThompson/Desktop/June'
for filename in os.listdir(folder):
    plpd=pd.DataFrame(columns=['SKU', 'ASIN', 'Product Title', 'Units Sold', 'Units Returned & Replaced', 'Per Unit Cost', 
                    'Total Production Cost', 'Product Sales', 'Shipping & Gift Wrap Fees','Referral Fees',
                    'FBA Packaging Fees', 'Promotional Rebates', 'Shipping, Gift Wrap, & Promo Chargebacks & Credits', 'Promo Rebate Refunds','Promotional Shipping Rebate Refunds',
                    'Product Sales Refunds' ,'Refund Processing & Restocking Fees',
                    'Shipping & Gift Wrap Chargeback & Credits Reimbursements','Reversal Reimbursements (Adjustments)', 
                    'Inbound Carrier Shipping Fees', 'Inventory Storage Fees', 'Inventory Placement, Prep & Removal Fees', 
                    'Inventory Damage & Loss Reimbursements',   'Advertising Spend', 'Subscription & Service Fees', 
                    'AWS Income', 'AWS Expense', 'CK Fees', 'Margin $', 'Margin %',  'Margin $ w/CK', 'Margin % w/CK'])

    filename=folder+ '/'+ filename
    client=filename.split('June/')[-1]
    client=client.split('pandl')[0]
    print(client)
    plpd, storage, orders=createindex(filename, plpd)
    plpd=plpd.set_index('SKU')

    sales, taxfee, sellfee, shipchargeback, giftchargeback, shipping, gift, promo,shippromo, refundNA, fbapickfee , shiptax, gifttax, shipcom, restockna= simplifycode(filename, 'Orders & FBA Commissions', paydet, 2)
    salesrefund, taxrefund, sellfeerefund, shippingchargebackrefund, giftchargebackrefund, shippingrefund, giftrefund, promorefund, shippromorefund, fbafeerefund, refundadmin, shiptaxrefund, gifttaxrefund, shipcomref, restock= simplifycode(filename, 'Refunds', paydet,9)
    replace, adspend ,invdamage, reversal, brandspend =replacereverse(filename)
    plpd, totltstore = servicefees(filename, plpd)
    refundpd=pd.read_excel(filename, sheet_name='Refunds')
    refundother=np.sum(refundpd.loc[refundpd['Payment Detail']=='Other concession', 'Amount'])
    plpd.loc['Other concession', 'Product Sales Refunds']=refundother
#   shiprefundother=np.sum(refundpd.loc[refundpd['SKU']=='', 'Amount'])
#    print(shiprefundother)
    for sku in plpd.index:
        rev=0
        units=0
        if sku in sales.index:
            rev=sales.loc[sku, 'Amount']
            units=sales.loc[sku, 'Quantity']
    
        plpd.loc[sku, 'Units Sold']=units
        plpd.loc[sku, 'Product Sales']=rev
        
        skupromo=0
        if sku in promo.index:
            skupromo=promo.loc[sku, 'Amount']
        plpd.loc[sku, 'Promotional Rebates']=skupromo
        
        skupromoship=0
        if sku in shippromorefund.index:
            skupromoship=shippromorefund.loc[sku, 'Amount']
        plpd.loc[sku, 'Promotional Shipping Rebate Refunds']=skupromoship
       
        skupickfee=0
        if sku in fbapickfee.index:
            skupickfee=fbapickfee.loc[sku, 'Amount']+skupickfee
        if sku in shipchargeback.index:
             skupickfee=shipchargeback.loc[sku, 'Amount']+skupickfee
        if sku in giftchargeback.index:
             skupickfee=giftchargeback.loc[sku, 'Amount']+skupickfee
        plpd.loc[sku, 'FBA Packaging Fees']=skupickfee
        
        skureturnfee=0
        if sku in refundadmin.index:
            skureturnfee=refundadmin.loc[sku, 'Amount']+skureturnfee
        if sku in fbafeerefund.index:
            skureturnfee=fbafeerefund.loc[sku, 'Amount']+skureturnfee
        if sku in refundNA.index:
            skureturnfee=refundNA.loc[sku, 'Amount']+skureturnfee
        plpd.loc[sku, 'Refund Processing & Restocking Fees']=skureturnfee
       
        skureplace=0
        if sku in replace.index:
            skureplace=replace.loc[sku, 'quantity']   
        skurefunds=0
        unitsrefunds=0
        if sku in salesrefund.index:
            skurefunds=salesrefund.loc[sku, 'Amount']+skurefunds
            unitsrefunds=salesrefund.loc[sku, 'Quantity']
        if sku in restock.index:
            skurefunds=restock.loc[sku, 'Amount']+skurefunds       
   
        plpd.loc[sku, 'Product Sales Refunds']=skurefunds
        plpd.loc[sku, 'Units Returned & Replaced']=float(-unitsrefunds+skureplace)
        
        skusellfee=0
        if sku in shipcom.index:
            skusellfee=shipcom.loc[sku, 'Amount']+skusellfee
        if sku in shipcomref.index:
            skusellfee=shipcomref.loc[sku, 'Amount']+skusellfee
        if sku in sellfee.index:
            skusellfee=sellfee.loc[sku, 'Amount']+skusellfee
        if sku in sellfeerefund.index:
            skusellfee=sellfeerefund.loc[sku, 'Amount']+skusellfee
        plpd.loc[sku, 'Referral Fees']=skusellfee
        
        skushipping=0
        if sku in shipping.index:
            skushipping=shipping.loc[sku, 'Amount']+skushipping
        if sku in gift.index:
            skushipping=gift.loc[sku, 'Amount']+skushipping
        plpd.loc[sku, 'Shipping & Gift Wrap Fees']=float(-skushipping)
        
        taxchargeback=0
        if sku in shiptax.index:
            taxchargeback=shiptax.loc[sku, 'Amount']+taxchargeback
        if sku in gifttax.index:
            taxchargeback=gifttax.loc[sku, 'Amount']+taxchargeback
        if sku in shiptaxrefund.index:
            taxchargeback=shiptaxrefund.loc[sku, 'Amount']+taxchargeback
        if sku in gifttaxrefund.index:
            taxchargeback= gifttaxrefund.loc[sku, 'Amount']+taxchargeback
        promoskuref=0
        if sku in promorefund.index:
            promoskuref=promorefund.loc[sku, 'Amount']
   
        plpd.loc[sku, 'Shipping, Gift Wrap, & Promo Chargebacks & Credits']=-float(-skushipping-promoskuref)
        plpd.loc[sku, 'Promo Rebate Refunds']=float(promoskuref)
        
        fbacomm=0
        if sku in shippingrefund.index:
             fbacomm=-shippingrefund.loc[sku, 'Amount']+fbacomm
        print(fbacomm)
        if sku in giftrefund.index:
             fbacomm=-giftrefund.loc[sku, 'Amount']+fbacomm
        print(fbacomm)
       
        plpd.loc[sku, 'Shipping & Gift Wrap Chargeback & Credits Reimbursements']=-float(fbacomm)
        if sku != 'Payment Report Total':
            adsku=0
            store=0
        else:
            adsku=-plpd.loc['Payment Report Total', 'Advertising Spend']
            store=-plpd.loc['Payment Report Total', 'Inventory Storage Fees']
        if sku in adspend.index:
            adsku=adspend.loc[sku, 'Spend']
        plpd.loc[sku, 'Advertising Spend']=-adsku

        if sku in storage.index:
            store=storage.loc[sku, 'estimated-monthly-storage-fee']
        plpd.loc[sku, 'Inventory Storage Fees']=float(-store+totltstore)
        damagesku=0
        if sku in invdamage.index:
            damagesku=invdamage.loc[sku, 'amount-total']
        plpd.loc[sku, 'Inventory Damage & Loss Reimbursements']=damagesku
        reversesku=0
        if sku in reversal.index:
            reversesku=reversal.loc[sku, 'amount-total']
        plpd.loc[sku,'Reversal Reimbursements (Adjustments)']=reversesku
            
    totlist=['Units Sold', 'Units Returned & Replaced', 'Product Sales', 'Shipping & Gift Wrap Fees','Referral Fees',
                    'FBA Packaging Fees', 'Promotional Rebates', 'Shipping, Gift Wrap, & Promo Chargebacks & Credits', 'Promo Rebate Refunds','Promotional Shipping Rebate Refunds',
                    'Product Sales Refunds' ,'Refund Processing & Restocking Fees',
                    'Shipping & Gift Wrap Chargeback & Credits Reimbursements','Reversal Reimbursements (Adjustments)', 
                    'Inventory Storage Fees', 'Inventory Damage & Loss Reimbursements',   'Advertising Spend' ]
    
    for col in totlist:
        totvalue=np.sum(plpd[col])
        if col != 'Inventory Storage Fees' and col!= 'Advertising Spend' and col!='Product Sales Refunds':
            
            plpd.loc['Payment Report Total',col]=totvalue
            plpd.loc['Payment Total', col]=totvalue
        if col =='Advertising Spend' or col =='Inventory Storage Fees':     
            reportval=plpd.loc['Payment Report Total',col]
            print(reportval)
            plpd.loc['Payment Total', col]=float(totvalue-reportval-brandspend)
        if col =='Product Sales Refunds':
            plpd.loc['Payment Report Total',col]=float(totvalue)
            plpd.loc['Payment Total', col]=float(totvalue)
        

            

    filenamefinal= '/Users/JoyThompson/Desktop/June Out/pandloutv38'+ client+ '.xlsx'
    writer = pd.ExcelWriter(filenamefinal, engine='xlsxwriter')
    plpd.to_excel(writer, sheet_name='lifetime')
    writer.save()

3BMedical
-526.61
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
-539.2099999999999
0
BHF
-1297.83
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
31.91
31.91
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


ValueError: Incompatible indexer with Series